## Anomaly Detection with Negative Selection

### Project description:

In this project, you will use the same Textor algorithm from the previous project to detect anomalies in a different data set. We will use a fetal monitoring dataset (known as a cardiotocography) from the UC Irvine Machine Learning Repository. 

### Project goals:

1. Apply the Textor algorithm to a practical example
2. Use ROC analysis to score the detector

### Project question overview:

1. Write a function to process the cardiotocography data set. [Question 1](#question1)
2. Paste any previous functions or write additional helper functions (not-graded). [Question 2](#question2)
3. Write a function to calculate AUC values over an interval of $r$ values. [Question 3](#question3)


## Cardiotocography Dataset

A cardiotocography is a technical way to measure the fetal heart rate (FHR) and the uterine contractions (UC) during pregnancy. Obstetricians then classify these readings as either normal, suspect, and pathologic. Figure 1 shows the display of a cardiotocograph (CTG), the FHR is shown in orange and the UC is shown in green. Figure 2 shows the output of a typical CTG where the line labeled **A** is the FHR and the line labeled **D** is the UC. More details on the data set are here: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/. 

The data consists of 21 real-valued variables, outliers, and inliers. Each row is a sample, each column is an observed variable, and the training file includes the ground truth labels of the samples. The data are in: ``cardio_train.csv`` and ``cardio_test.csv``. 

<table><tr>
<td> 
  <p align="center" style="padding: 10px">
    <img alt="Forwarding" src="images/cardio.jpg" width="250">
    <br>
    <em style="color: grey">Figure 1: Cardiotocograph display</em>
  </p> 
</td>
<td> 
  <p align="center">
    <img alt="Routing" src="images/CTG_Output.jpg" width="650">
    <br>
    <em style="color: grey">Figure 2: Cardiotocograph output</em>
  </p> 
</td>
</tr></table>

*Image sources: Wikipedia*



## Data Processing

In order for the continuous data to work well with the negative selection algorithm, we will need to bin the values into categorical variables. For consistency across assignments, we will simply bin each of the variables into 10 bins. This will produce strings of length 21 symbols and an alphabet size of 10 characters (e.g. A indicates the lowest category and J indicates the highest). 

Next, we will need to format the data in such a way that the new train and test data may be read directly by Textor's algorithm used in the previous project. Recall that both the train and the test data for the languages were ``.csv`` files with one string of length 10 per line. Here, we would like there to be one string of length 21 per line. These strings will correspond to the 21 variables in a row of either ``cardio_train.csv`` or ``cardio_test.csv``. 

### Creating Bins with pandas

To avoid ambiguity in how the data can be binned, we suggest that you use the ``cut`` function from pandas. Note that this function allows for a label parameter to be passed. To map our binned data to characters we can define the labels array as follows:

``labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']``.

After defining labels we can call ``pandas.cut(x, bins=10, labels)``, where ``x`` is the data column.

<a id='question1'></a>
# Question 1

Write the function ``process_data(file)`` which processes either ``cardio_train.csv`` or ``cardio_test.csv`` into the format specified in the "Data Processing" section. This function returns the string ``processed_file`` which is a path to the final processed file. This function will not be directly graded but will be used in the grading of later questions. The processed file should be in ``.csv`` format with one string of length 21 per row. The final processed file should look similar to the strings below. 

**Do not include headers or indices in your final processed file. The file should contain only strings.**

```
EEDAECEAACACIBJDBEDDA
EEBAFCEAACACIBJCBEDDA
EEBAFBEAADAEHAFGAEDDA
EEDAFAEAADADHAFFAEDDA
EECBFBEACBACDDDCAEDDA
EEEEEAEACBABFBEBAEDEA
DDDICDECBCABGBGDAFCDE
DDCHCCEEBCABGBGBAECDC
DDDJDCEABCACIAHCAFCDD
DDBJDCEFBCABGAEDADCCC
DDCJDCEDCBABHBGDBDCDB
DDCICCEFCBABJAJFADCCB
EEACAHEACIAAJAJFADAAF
...
```


In [125]:
# import csv
import numpy as np
from subprocess import run, PIPE, TimeoutExpired
from sklearn.metrics import auc
import pandas as pd

def process_data(file):
    """
    Process data from CSV into binned strings of length 21 using pure Python.
    """
    labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    processed_rows = []

    df = pd.read_csv(file)
    # Iterate over rows by name
    for index, row in df.iterrows():
        # Access the column's values
        row_values = row[1:22]
        binned_row = pd.cut(row_values, bins=10, labels=labels, right=False)
        processed_rows.append(''.join(binned_row))  

    # # Read file
    # with open(file, 'r') as f:
    #     reader = csv.reader(f)
    #     for row in reader:
    #         try:
    #             # Convert all values in the row to floats
    #             row = [float(value) for value in row]
    #             row = row[1:]
    #             # Bin each column value into one of 10 categories
    #             binned_row = pd.cut(row, bins=10, labels=labels)
    #             processed_rows.append(''.join(binned_row))  # Ensure 21 characters per line
    #         except ValueError:
    #             # Skip rows that cannot be converted to floats
    #             print(f"Skipping invalid row: {row}")
    #             continue

    # Save processed file
    processed_file = file.replace('.csv', '_processed.csv')
    with open(processed_file, 'w') as f:
        for line in processed_rows:
            f.write(line + '\n')

    print(f"Processed file saved as: {processed_file} with {len(processed_rows)} rows.")
    return processed_file


## Implementing Negative Selection Recap

Recall from the previous project that we used a Python subprocess to use Textor's ``negsel.jar`` file. To start a subprocess, we will first need to import the ``subprocess`` module and the ``run`` and ``PIPE`` submodules. Then, we will need to run the process with an array of arguments and the opened input file. We will use the Textor example ([here](http://johannes-textor.name/negativeselection.html)) to demonstrate how subprocesses work.

The following command trains the negative selection algorithm on the ``english.train`` data set and tests on the ``hiligaynon.test`` data set from the previous project.

``java -jar negsel.jar -c -n 10 -r 4 -self english.train < hiligaynon.test ``

In this project, you will use the processed train and test data to run this same command. Recall that to run this subprocess, we will need to first build an argument list from the command. This allows the subprocess command to parse the command and flags. Using the command given above, we will construct the following argument list. 

``args = ['java', '-jar', 'negsel.jar', '-c', '-n', '10', '-r','4', '-self', 'negsel_src/tests/english.train']``

Next, we will need to open the test file to read from. This step is necessary as the argument list will not recognize the redirection from stdin (``< hiligaynon.test``).

``input_file = open('negsel_src/tests/hiligaynon.test', 'r')``

Finally, we can use the ``run`` submodule to call the command and pipe the output. 

``result = run(args, stdout=PIPE, stdin=input_file, universal_newlines=True)``

The final results will be stored in ``result.stdout``. This can be saved and parsed later as needed. 

Refer to the previous project for more information on how to run the code as well as a full running example.

**REMEMBER TO ADJUST THE STRING LENGTH FOR THIS PROJECT.**

## Exploring Model Parameterization

In the previous project, we used $r=4$ contiguous bits for all of our testing. However, this does not ensure optimal anomaly detection. In this project, we will investigate the tuning of the $r$ parameter for the *$r$-contiguous* patterns. 
 

### $r$-contiguous Patterns

Textor in _"A Comparative Study of Negative Selection Based Anomaly Detection in Sequence Data"_ defines the $r$-contiguous pattern as follows.

<blockquote>An $r$-contiguous pattern is a string $d\in \Sigma^l$. It matches another string $s\in  \Sigma^l$ if $d$ and $s$ are identical in at least $r$ contiguous positions, i.e., if there is an $i\in \{1,...,l-r+1\}$ such that the substrings of length $r$ of $s$ and $d$ starting at the $i$-th position are equal. </blockquote> 

### Scoring the Detector 

Since our strings are of length 21, we will consider $r\in [2,10]$. For simplicity, we will use the *AUC* as described in the previous project. Recall that the ROC curve is created by plotting the false positive (FP) rate and the true positive (TP) rate over an interval of values of $\theta$. Generally, a meaningful classifier has an area under the curve (AUC) value greater than 0.5, and an AUC value close to 1 signifies a near-perfect classifier. 

You may use the ``fp_tp_calc(file, theta)`` and ``auc_calc(file)`` functions from the previous project to calculate the AUC values. Additionally, we will let $\theta \in [0,10]$, and $ \theta \in \mathbb{Z}$. 

<a id='question2'></a>
# Question 2: Using previous functions

Use the answer box below to paste in any functions that you would like to use for Question 2. You may also use this area to write any additional helper functions and/or edit your old functions to generalize to this problem and dataset (i.e., by adding a new parameter to handle the fact that r is an additional input variable). Labels for this dataset may be found in the file cardio.labels.


In [126]:
from sklearn import metrics
import subprocess

train_file = process_data('cardio_train.csv')

# Step 2: FP and TP calculation
def fp_tp_calc( file, theta, r):
    args = ['java', '-jar', 'negsel.jar', '-c', '-n', '21', '-r', str(r), '-self', train_file]
    result = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=file, universal_newlines=True)
    scores = [float(line) for line in result.stdout.strip().splitlines()]

    labels = pd.read_csv('cardio.labels', header=None).squeeze().tolist()
    
    tp = sum(1 for i in range(len(scores)) if scores[i] > theta and labels[i] == 1)
    fn = sum(1 for i in range(len(scores)) if scores[i] <= theta and labels[i] == 1)
    fp = sum(1 for i in range(len(scores)) if scores[i] > theta and labels[i] == 0)
    tn = sum(1 for i in range(len(scores)) if scores[i] <= theta and labels[i] == 0)
    
    fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
    return (fpr, tpr)

def auc_calc(file, r):
    theta_range = range(0, 11)
    print("calculating value for r:" + str(r))
    input_file = open(process_data(file), 'r')
    fpr_tpr = [fp_tp_calc(input_file, theta, r) for theta in theta_range]
    
    fprs, tprs = zip(*fpr_tpr)
    auc = metrics.auc(fprs, tprs)
    return round(auc, 2)

Processed file saved as: cardio_train_processed.csv with 1000 rows.


<a id='question3'></a>
# Question 3

Write the function ``r_auc_tuple(file)`` that calculates a list of tuples for each value of $r\in [2,10]$ over $\theta \in [0,10]$ for a given test file. This will result in a list of 10 tuples where the first element in each tuple is the value of $r$ and the second element is the AUC value.  This list will be returned as ``tuples_list`` and will be graded based on accuracy. The labels for the test file are in ``cardio.labels``.

For reference, the following command should return the AUC value for $r=2$: ``r_auc_tuple('cardio_test.csv')[0][1]``.

**Remember to use the ``process_data(file)`` function to run the Textor algorithm with the correct files.**


In [127]:
def r_auc_tuple(file):
    """
    Calculate AUC values for various values of r (2 to 10) for a given test file.
    """
    # train_file = process_data('cardio_train.csv')
    # test_file = process_data(file)

    # # Read labels
    # with open('cardio.labels', 'r') as f:
    #     labels = [int(line.strip()) for line in f]

    # # Debugging: Check lengths of test file and labels
    # print(f"Number of labels: {len(labels)}")

    tuples_list = []

    for r in range(2, 5):
        # timeout = 120
        # args = ['java', '-jar', 'negsel.jar', '-c', '-n', '10', '-r', str(r), '-self', train_file]

        # with open(test_file, 'r') as input_file:
        #     try:
        #         result = run(args, stdout=PIPE, stdin=input_file, universal_newlines=True, timeout=timeout)
        #     except TimeoutExpired:
        #         print(f"Timeout expired for r={r}, skipping to the next value.")
        #         continue

        # # Parse scores from negsel.jar output
        # scores = []
        # for line in result.stdout.splitlines():
        #     for value in line.split():
        #         try:
        #             scores.append(float(value))
        #         except ValueError:
        #             continue

        # # Debugging: Check scores
        # print(f"Scores for r = {r}: {scores[:10]} (first 10 scores)")

        # if not scores:
        #     print(f"No scores generated for r = {r}")
        #     continue

        # # Normalize scores
        # max_score = max(scores)
        # normalized_scores = [(score / max_score) * 50 for score in scores] if max_score > 0 else [0] * len(scores)

        # # Debugging: Check normalized scores
        # print(f"Normalized Scores for r = {r}: {normalized_scores[:10]} (first 10 normalized)")

        # # Ensure scores and labels align
        # effective_length = min(len(normalized_scores), len(labels))
        # normalized_scores = normalized_scores[:effective_length]
        # labels = labels[:effective_length]

        # # Debugging: Check alignment
        # print(f"Effective lengths for r = {r}: {effective_length}")

        # # Calculate FP-TP values
        # theta_range = np.arange(0, 11, 1)
        # fp_tp_values = [fp_tp_calc(normalized_scores, theta, labels) for theta in theta_range]

        # # Debugging: Check FP-TP values
        # print(f"FP-TP Values for r = {r}: {fp_tp_values}")

        # if len(fp_tp_values) < 2:
        #     print(f"Insufficient FP-TP values for r = {r}, skipping.")
        #     continue

        # Calculate AUC
    
        auc_value = auc_calc(file, r)
        tuples_list.append((r, auc_value))
       

    return tuples_list


In [128]:
'''
You must run this cell.
Check the output of your functions.
'''
import csv

tuples = r_auc_tuple('cardio_test.csv')
print(tuples)

# don't edit the below code
with open('tuples.csv','w',newline='') as file:
    writer = csv.writer(file)
    writer.writerows(tuples)

assert len(tuples) == 9
assert round(tuples[0][1],2) == 0.75
assert round(tuples[7][1],2) == 0.0
print("All test cases passed")

calculating value for r:2
Processed file saved as: cardio_test_processed.csv with 831 rows.
calculating value for r:3
Processed file saved as: cardio_test_processed.csv with 831 rows.
calculating value for r:4
Processed file saved as: cardio_test_processed.csv with 831 rows.
[(2, np.float64(0.0)), (3, np.float64(0.42)), (4, np.float64(0.5))]


AssertionError: 